In [1]:
!kaggle datasets download -d romainpessia/artificial-lunar-rocky-landscape-dataset

Dataset URL: https://www.kaggle.com/datasets/romainpessia/artificial-lunar-rocky-landscape-dataset
License(s): CC-BY-NC-SA-4.0
100% 5.01G/5.02G [00:43<00:00, 153MB/s]
100% 5.02G/5.02G [00:43<00:00, 124MB/s]


In [2]:
import zipfile
zip_ref = zipfile.ZipFile('/content/artificial-lunar-rocky-landscape-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2 as cv
from keras.models import Model,load_model
from keras.layers import Input, Conv2D, Conv2DTranspose,AveragePooling2D, MaxPooling2D,UpSampling2D,LeakyReLU, concatenate, Dropout,BatchNormalization,Activation
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pandas as pd
from keras.optimizers import Adam
InputPath = "/content/images/"


In [4]:
def Generator(X, y, batch_size=1):
    while True:
        img_batch_1 = []
        img_batch_2 = []

        for _ in range(batch_size):
            i = np.random.randint(0, len(X))

            img_1 = cv.imread(InputPath + 'render/' + X[i])
            img_1 = cv.resize(img_1, (512, 512))
            img_1 = img_1 / 255.0

            img_2 = cv.imread(InputPath + 'ground/' + y[i])
            img_2 = cv.resize(img_2, (512, 512))
            img_2 = img_2 / 255.0

            img_batch_1.append(img_1)
            img_batch_2.append(img_2)

        yield np.array(img_batch_1), np.array(img_batch_2)

In [5]:
SourceImg = sorted(os.listdir(InputPath+'render'))
TargetImg = sorted(os.listdir(InputPath+'ground'))
Gen = Generator(SourceImg,TargetImg)

In [ ]:
next(Gen)[0].shape,next(Gen)[1].shape

((1, 512, 512, 3), (1, 512, 512, 3))

In [ ]:
VGG16 = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(512, 512,3))
VGG16.trainable = False
for layer in VGG16.layers:
  if layer.name.startswith('block5'):
    layer.trainable = True
  print(layer.name,layer.trainable,layer.output.shape)
last_layer = VGG16.output

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
input_layer False (None, 512, 512, 3)
block1_conv1 False (None, 512, 512, 64)
block1_conv2 False (None, 512, 512, 64)
block1_pool False (None, 256, 256, 64)
block2_conv1 False (None, 256, 256, 128)
block2_conv2 False (None, 256, 256, 128)
block2_pool False (None, 128, 128, 128)
block3_conv1 False (None, 128, 128, 256)
block3_conv2 False (None, 128, 128, 256)
block3_conv3 False (None, 128, 128, 256)
block3_pool False (None, 64, 64, 256)
block4_conv1 False (None, 64, 64, 512)
block4_conv2 False (None, 64, 64, 512)
block4_conv3 False (None, 64, 64, 512)
block4_pool False (None, 32, 32, 512)
block5_conv1 True (None, 32, 32, 512)
block5_conv2 True (None, 32, 32, 512)
block5_conv3 True (None, 32, 32, 512)
block5_pool True (None, 16, 16, 512)


In [ ]:
model = Conv2DTranspose(256,(3,3),strides=(2, 2),padding='same')(last_layer)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

concat_1 = concatenate([model,VGG16.get_layer("block5_conv3").output])

model = Conv2D(512,(3,3),strides=(1, 1),padding='same')(concat_1)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Conv2DTranspose(512,(3,3),strides=(2, 2),padding='same')(model)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

concat_2 = concatenate([model,VGG16.get_layer("block4_conv3").output])

model = Conv2D(512,(3,3),strides=(1, 1),padding='same')(concat_2)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Conv2DTranspose(512,(3,3),strides=(2, 2),padding='same')(model)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

concat_3 = concatenate([model,VGG16.get_layer("block3_conv3").output])

model = Conv2D(256,(3,3),strides=(1, 1),padding='same')(concat_3)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Conv2DTranspose(256,(3,3),strides=(2, 2),padding='same')(model)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

concat_4 = concatenate([model,VGG16.get_layer("block2_conv2").output])

model = Conv2D(128,(3,3),strides=(1, 1),padding='same')(concat_4)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Conv2DTranspose(128,(3,3),strides=(2, 2),padding='same')(model)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

concat_5 = concatenate([model,VGG16.get_layer("block1_conv2").output])

model = Conv2D(64,(3,3),strides=(1, 1),padding='same')(concat_5)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Conv2D(32,(3,3),strides=(1, 1),padding='same')(model)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Conv2D(3,(3,3),strides=(1, 1),padding='same')(model)
model = LeakyReLU(0.1)(model)
model = BatchNormalization()(model)

model = Model(VGG16.input,model)

In [ ]:
model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
callback = ModelCheckpoint('UNET.keras', verbose=1,mode='auto', monitor='loss',save_best_only=True)

In [ ]:
model.fit(Gen,epochs=500,callbacks=[callback],steps_per_epoch=len(SourceImg),shuffle=True)

Epoch 1/30
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.4929 - loss: 0.4110
Epoch 1: loss improved from inf to 0.24264, saving model to UNET.keras
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 2482s 250ms/step - accuracy: 0.4929 - loss: 0.4110
Epoch 2/30
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.4807 - loss: 0.1251
Epoch 2: loss improved from 0.24264 to 0.12159, saving model to UNET.keras
9766/9766 ━━━━━━━━━━━━━━━━━━━━ 2433s 249ms/step - accuracy: 0.4807 - loss: 0.1251
Epoch 3/30
9073/9766 ━━━━━━━━━━━━━━━━━━━━ 2:52 249ms/step - accuracy: 0.4736 - loss: 0.1129

In [ ]:
model.save('UNET.h5')

In [ ]:
callback = ModelCheckpoint('UNET.keras', verbose=1,mode='auto', monitor='loss',save_best_only=True)
from tensorflow.keras.models import load_model
model = load_model('UNET.keras')
model.fit(Gen,epochs=15,callbacks=[callback],steps_per_epoch=len(SourceImg),shuffle=True)

Epoch 1/15
6981/9766 ━━━━━━━━━━━━━━━━━━━━ 12:14 264ms/step - accuracy: 0.4611 - loss: 0.1158